In [ ]:
import argparse
import os
import random

import numpy as np
import torch
import gc
# import torch.backends.cudnn as cudnn

import gradio as gr

from ad_invento.common.config import Config
from ad_invento.common.dist_utils import get_rank
from ad_invento.common.registry import registry
from ad_invento.conversation.conversation import Chat, CONV_VISION, StoppingCriteriaSub

# imports modules for registration
from ad_invento.datasets.builders import *
from ad_invento.models import *
from ad_invento.processors import *
from ad_invento.runners import *
from ad_invento.tasks import *

import argparse
import time
from PIL import Image

import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
from transformers import StoppingCriteriaList

# import dataclasses
# from enum import auto, Enum
# from typing import List, Tuple, Any


def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    cudnn.benchmark = False
    cudnn.deterministic = True

def create_custom_args(cfg_path, gpu_id, options=None):
    args = argparse.Namespace(cfg_path=cfg_path, gpu_id=gpu_id, options=options)
    return args

# Example values for cfg_path and gpu_id
custom_cfg_path = "eval_configs/invento_eval.yaml"
custom_gpu_id = 0


args = create_custom_args(custom_cfg_path, custom_gpu_id)
cfg = Config(args)

model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id)) # 'cuda:{}'.format(args.gpu_id)

vis_processor_cfg = cfg.datasets_cfg.cc_sbu_align.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)


for name, param in model.visual_encoder.named_parameters():
    param.requires_grad=False
    
for name, param in model.visual_encoder_proj.named_parameters():
    param.requires_grad=False

for name, param in model.Qformer.named_parameters():
    param.requires_grad=False
    
for name, param in model.llama_model.named_parameters():
    param.requires_grad=False
    
for name, param in model.llama_proj.named_parameters():
    param.requires_grad=False

model.visual_encoder.eval()
model.visual_encoder_proj.eval()
model.Qformer.eval()
model.llama_model.eval()
model.llama_proj.eval()


device = torch.cuda.current_device()
gc.collect(device)
torch.cuda.empty_cache()

chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id)) # device='cuda:{}'.format(args.gpu_id)
from PIL import Image

img_path = 'F:\\Data_kaggle\\images\\images_normalized\\1013_IM-0013-1001.dcm.png'
image = Image.open(img_path).convert("RGB")

def gradio_reset(chat_state, img_list):
    if chat_state is not None:
        chat_state.messages = []
    if img_list is not None:
        img_list = []
    return None, gr.update(value=None, interactive=True), gr.update(placeholder='Please upload your image first', interactive=False),gr.update(value="Upload & Start Chat", interactive=True), chat_state, img_list

def upload_img(gr_img, text_input, chat_state):
    if gr_img is None:
        return None, None, gr.update(interactive=True), chat_state, None
    chat_state = CONV_VISION.copy()
    img_list = []
    llm_message = chat.upload_img(gr_img, chat_state, img_list)
    return gr.update(interactive=False), gr.update(interactive=True, placeholder='Type and press Enter'), gr.update(value="Start Chatting", interactive=False), chat_state, img_list, llm_message

def gradio_ask(user_message, chatbot, chat_state):
    if len(user_message) == 0:
        return gr.update(interactive=True, placeholder='Input should not be empty!'), chatbot, chat_state
    chat.ask(user_message, chat_state)
    chatbot = chatbot + [[user_message, None]]
    return '', chatbot, chat_state


def gradio_answer(chatbot, chat_state, img_list, num_beams, temperature):
    llm_message = chat.answer(conv=chat_state,
                              img_list=img_list,
                              num_beams=num_beams,
                              temperature=temperature,
                              max_new_tokens=300,
                              max_length=64)[0]
    chatbot[-1][1] = llm_message
    return chatbot, chat_state, img_list

x2 = upload_img(gr_img=image, text_input="Did you find any abnormality ?", chat_state=0 )
embs = x2[4][0]

vis_processor = vis_processor
device = 'cuda'
stop_words_ids = [torch.tensor([835]).to(device),
                  torch.tensor([2277, 29937]).to(device)]  # '###' can be encoded in two different ways.
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

device = torch.cuda.current_device()
gc.collect(device)
torch.cuda.empty_cache()

outputs = model.llama_model.generate(
            inputs_embeds=embs,
#             max_new_tokens=300,
            stopping_criteria=stopping_criteria,
            num_beams=5,
            num_return_sequences=4,
            do_sample=True,
            min_length=20,
            max_length=64,
            top_p=0.8,
            top_k=5000,
            repetition_penalty=2.0,
            length_penalty=1.0,
            temperature=1,
            no_repeat_ngram_size=2,
        )

output_token = outputs[0]
if output_token[0] == 0:  # the model might output a unknow token <unk> at the beginning. remove it
    output_token = output_token[1:]
if output_token[0] == 1:  # some users find that there is a start token <s> at the beginning. remove it
    output_token = output_token[1:]
output_text = model.llama_tokenizer.decode(output_token, add_special_tokens=False)
output_text = output_text.split('###')[0]  # remove the stop sign '###'
output_text = output_text.split('Assistant:')[-1].strip()

print("Output Text -> ", output_text)